## **Import thư viện cần thiết**

In [7]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random

from PIL import Image
import copy
import time

import os
os.environ["NO_ALBUMENTATIONS_UPDATE"] = "0, 1"

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from torchsummary import summary
from sklearn.metrics import confusion_matrix, classification_report

import torch.nn.functional as F

import albumentations as A
from albumentations.pytorch import ToTensorV2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


## **Data Augmentation using Albumentations¶**

In [ ]:
data_transforms = {
    'train': A.Compose([
        A.Resize(224, 224),
        A.RandomCrop(224, 224),
        A.HorizontalFlip(),
        A.RandomBrightnessContrast(),
        A.Rotate(limit = 15),
        A.CoarseDropout(max_holes = 1, max_height = 16, max_width = 16, fill_value = 0, mask_fill_value=None),
        A.Normalize(mean= (0.485, 0.456, 0.406), std= (0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]),
    'val': A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]),
    
    'test': A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
}

## **Custom Dataset Loader**

In [ ]:
class CustomImageDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None):
        super(CustomImageDataset, self).__init__(root, transform=None)
        self.custom_transform = transform

    def __getitem__(self, index):
        image, label = super(CustomImageDataset, self).__getitem__(index)
        image = np.array(image)
        if self.custom_transform:
            augmented = self.custom_transform(image = image)
            image = augmented['image']
        return image, label

## **Dataset Loading and Splitting**

In [ ]:
data_dir = '/kaggle/input/dataset-processing-equalhist-filternoise-edges/brain-tumor-mri-equalHist-FilterNoise-Edges-dataset'
train_dataset = CustomImageDataset(data_dir + "/Training", transform=data_transforms['train'])
val_test_dataset = CustomImageDataset(data_dir + "/Testing", transform=data_transforms['train'])

val_size = int(0.5 * len(val_test_dataset))
test_size = len(val_test_dataset) - val_size
val_dataset, test_dataset = random_split(val_test_dataset, [val_size, test_size])

val_dataset.dataset.custom_transform = data_transforms['val']
test_dataset.dataset.custom_transform = data_transforms['test']

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

## **Model Architecture**

In [ ]:
# Mô hình ResNet50
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)
model = model.to(device)

## **Loss function and Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

## **Training and Evaluation Functions¶**

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_eposhs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss':[], 'val_loss':[], 'val_acc':[]}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        val_loss, val_acc = evaluate_model(model, val_loader, criterion)
        scheduler.step(val_loss)
        print(f"Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}, Val Acc: {val_acc}")
        history['train_loss'].append(running_loss/len(train_loader))
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model, history

In [ ]:
def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / len(data_loader), 100 * correct / total

## **Training the model**

In [ ]:
num_epochs = 40
model, history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)

## **Evaluating the Model on the Val Set**

In [ ]:
test_loss, test_acc = evaluate_model(model, val_loader, criterion)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}%")

## **Evaluating the Model on the Test Set**

In [ ]:
test_loss, test_acc = evaluate_model(model, test_loader, criterion)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}%")

## **Training and Validation Metrics for Model Comparison**

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()
#=============================================================
plt.figure(figsize=(10, 5))
plt.plot(history['val_acc'], label='Validation Accuracy')
plt.title('Validation Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

## **Confusion Matrix and Classification Report**

In [ ]:
def plot_confusion_matrix(model, data_loader, classes):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    return all_preds, all_labels

In [ ]:
classes = ['glioma', 'meningioma', 'notumor', 'pituitary']
all_preds, all_labels = plot_confusion_matrix(model, test_loader, classes)
print('Classification Report:')
print(classification_report(all_labels, all_preds, target_names=classes))

In [ ]:
classes = ['glioma', 'healthy', 'meningioma', 'pituitary']
all_preds, all_labels = plot_confusion_matrix(model, test_loader, classes)
print('Classification Report:')
print(classification_report(all_labels, all_preds, target_names=classes))

## **Saving The Model**

In [ ]:
model_path = '/kaggle/working/results_model.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved at {model_path}')

In [ ]:
model_path = '/kaggle/working/results_model_NewDataset.pth'
torch.save(model.state_dict(), model_path)
print(f'Model saved at {model_path}')

## **Load Model**

In [4]:
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

model.load_state_dict(torch.load('/kaggle/working/results_model_NewDataset.pth'))
model = model.to(device)

/tmp/ipykernel_30/3546259920.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/working/results_model_NewDataset.pth'))


In [11]:
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
classes = ['glioma', 'notumor', 'meningioma', 'pituitary']
all_preds, all_labels = plot_confusion_matrix(model, test_loader, classes)
print('Classification Report:')
print(classification_report(all_labels, all_preds, target_names=classes))